```{contents}
```
## Background Tasks


**Background tasks** are operations that run **asynchronously** after an API request is returned to the user, without blocking the main request–response cycle.

They are used for:

* Logging
* Sending emails / notifications
* Data processing
* Model training / indexing
* Long-running LLM jobs

---

### Why Background Tasks Are Needed

Without background tasks:

```
Client waits → Server blocks → Timeouts → Poor UX
```

With background tasks:

```
Client gets response immediately → Task runs in background
```

---

### Where They Fit in the System

```
Client Request
     ↓
API Handler → Return Response Immediately
     ↓
Background Task Executes Independently
```

---

### FastAPI Background Tasks (Native Support)

#### Demonstration

```python
from fastapi import FastAPI, BackgroundTasks

app = FastAPI()

def write_log(message: str):
    with open("log.txt", "a") as f:
        f.write(message + "\n")

@app.post("/process")
def process_task(data: str, background_tasks: BackgroundTasks):
    background_tasks.add_task(write_log, f"Processed: {data}")
    return {"status": "Task started"}
```

---

### Long-Running Task Example

#### Demonstration

```python
import time

def long_job(job_id):
    time.sleep(10)
    print(f"Job {job_id} completed")

@app.post("/start-job")
def start_job(job_id: int, background_tasks: BackgroundTasks):
    background_tasks.add_task(long_job, job_id)
    return {"message": "Job running in background"}
```

---

### Background Tasks with LLM Work

```python
def generate_embeddings(text):
    vector = embedding_model.encode(text)
    store_vector(vector)

@app.post("/upload")
def upload(text: str, background_tasks: BackgroundTasks):
    background_tasks.add_task(generate_embeddings, text)
    return {"status": "Indexing started"}
```

---

### When to Use External Workers

FastAPI background tasks are **in-process**.
For heavy or critical workloads use **Celery / RQ / Dramatiq**.

#### Example with Celery

```python
@celery.task
def process_large_file(file_id):
    ...
```

---

### Reliability Considerations

| Concern        | Solution            |
| -------------- | ------------------- |
| Task crash     | Retry logic         |
| Server restart | External queue      |
| High load      | Distributed workers |
| Tracking       | Job status store    |

---

### Mental Model

```
Background Tasks = Work done after the user has already been answered
```

---

### Key Takeaways

* Improves responsiveness dramatically
* Essential for production APIs
* FastAPI provides built-in support
* Use task queues for heavy workloads